# Imperative Demo - Product Catalog Service

This notebook demonstrates the traditional approach with:
- Terraform for infrastructure (S3 bucket + IAM)
- Kubernetes manifests for application deployment
- Dagger (Go) for workflow automation and testing

**Total: 746 lines across 6 files**
- Terraform: 244 lines
- K8s manifests: 196 lines
- Dagger pipeline: 306 lines Go

## Prerequisites Check

In [ ]:
# Verify cluster and tools
!kubectl get nodes
!terraform version
!go version
!k3d registry list

## Step 1: Review Infrastructure Code

Terraform manages AWS resources: S3 bucket and IAM role.

In [ ]:
# Show Terraform configuration
print("=== provider.tf ===")
!cat imperative/terraform/provider.tf

print("\n=== main.tf (excerpt) ===")
!cat imperative/terraform/main.tf | head -40

## Step 2: Review Kubernetes Manifests

Multiple YAML files for deployment, service, HPA, etc.

In [ ]:
# List all K8s manifests
!ls -lh imperative/k8s/

print("\n=== deployment.yaml (excerpt) ===")
!cat imperative/k8s/deployment.yaml | head -50

## Step 3: Review Dagger Pipeline

Go code orchestrates Terraform, Docker, K8s, and testing.

In [ ]:
# Show Dagger pipeline structure
!wc -l imperative/dagger/main.go

print("\n=== main.go (excerpt) ===")
!cat imperative/dagger/main.go | head -60

## Step 4: Initialize Terraform

Download providers and prepare infrastructure.

In [ ]:
%%bash
cd imperative/terraform
terraform init

## Step 5: Verify AWS Credentials

In [ ]:
# Test AWS connectivity
!set -a && source ../.env.aws && set +a && aws sts get-caller-identity

## Step 6: Build and Push Docker Image

Build the application container and push to k3d registry.

In [ ]:
%%bash
cd app
echo "Building Docker image..."
DOCKER_BUILDKIT=0 docker build -t imp-product-catalog:v1.0.0-imperative .

echo "Tagging for local registry..."
docker tag imp-product-catalog:v1.0.0-imperative localhost:5000/imp-product-catalog:v1.0.0-imperative

echo "Pushing to k3d registry..."
docker push localhost:5000/imp-product-catalog:v1.0.0-imperative

echo "✓ Image available at k3d-registry.localhost:5000 from cluster"

## Step 7: Create S3 Bucket with Terraform

Provision AWS infrastructure.

In [ ]:
%%bash
cd imperative/terraform

if [ -f "../../../.env.aws" ]; then
    echo "Loading AWS credentials from .env.aws..."
    source ../../../.env.aws
    export AWS_ACCESS_KEY_ID
    export AWS_SECRET_ACCESS_KEY
    export AWS_SESSION_TOKEN
    export AWS_DEFAULT_REGION

    # Force Terraform to use environment variables
    unset AWS_PROFILE
    unset AWS_SDK_LOAD_CONFIG
    export AWS_CONFIG_FILE=/dev/null
    export AWS_SHARED_CREDENTIALS_FILE=/dev/null

    echo "✓ AWS credentials loaded"
    echo ""
else
    echo "Warning: .env.aws not found. AWS operations may fail."
    echo "Please create .env.aws in the kubecon-na-2025 directory"
    echo ""
fi

# Apply Terraform
terraform plan
terraform apply -auto-approve

## Step 8: Verify S3 Bucket Creation

In [ ]:
!set -a && source ../.env.aws && set +a && aws s3 ls | grep tenant-atlantis-product-images-imperative

## Step 9: Deploy to Dev Environment

Apply Kubernetes manifests to dev namespace.

In [ ]:
%%bash
# Create namespace
kubectl create namespace dev --dry-run=client -o yaml | kubectl apply -f -

# Apply all manifests
kubectl apply -f imperative/k8s/ -n dev

# Wait for deployment
kubectl rollout status deployment/imp-product-catalog -n dev --timeout=120s

## Step 10: Verify Dev Deployment

In [ ]:
!kubectl get pods,svc,hpa -n dev

## Step 11: Test the API in Dev

Port-forward and test the API endpoints.

In [ ]:
import subprocess
import time
import requests
import json

# First, check if pods are ready
print("Checking pod status...")
pod_check = subprocess.run(
      ["kubectl", "get", "pods", "-n", "dev", "-l", "app=imp-product-catalog"],
      capture_output=True,
      text=True
  )
print(pod_check.stdout)

  # Wait for pods to be ready
print("\nWaiting for pods to be ready...")
subprocess.run(
      ["kubectl", "wait", "--for=condition=ready", "pod", "-n", "dev",
       "-l", "app=imp-product-catalog", "--timeout=120s"],
      check=False
  )

  # Start port-forward in background
print("\nStarting port-forward...")
port_forward = subprocess.Popen(
      ["kubectl", "port-forward", "-n", "dev", "svc/imp-product-catalog", "8081:80"],
      stdout=subprocess.PIPE,
      stderr=subprocess.PIPE
  )

  # Give port-forward more time to establish
time.sleep(5)

  # Check if port-forward is still running
if port_forward.poll() is not None:
      stdout, stderr = port_forward.communicate()
      print(f"Port-forward failed to start:")
      print(f"stdout: {stdout.decode()}")
      print(f"stderr: {stderr.decode()}")
      raise Exception("Port-forward failed")

try:
      # Test health endpoint with retries
      print("Testing /health endpoint...")
      max_retries = 5
      for i in range(max_retries):
          try:
              response = requests.get("http://localhost:8081/health", timeout=5)
              print(f"Status: {response.status_code}")
              print(json.dumps(response.json(), indent=2))
              break
          except requests.exceptions.ConnectionError as e:
              if i < max_retries - 1:
                  print(f"Connection failed, retrying ({i+1}/{max_retries})...")
                  time.sleep(2)
              else:
                  raise

      # Create a product
      print("\nCreating a test product...")
      product_data = {
          "name": "Imperative Test Product",
          "description": "Created from traditional approach",
          "price": 99.99
      }
      response = requests.post("http://localhost:8081/products", json=product_data, timeout=10)
      print(f"Status: {response.status_code}")
      product = response.json()
      print(json.dumps(product, indent=2))

      # Retrieve the product
      print(f"\nRetrieving product {product['id']}...")
      response = requests.get(f"http://localhost:8081/products/{product['id']}", timeout=10)
      print(f"Status: {response.status_code}")
      print(json.dumps(response.json(), indent=2))

except Exception as e:
      print(f"\nError occurred: {str(e)}")
      print("\nTroubleshooting:")
      print("1. Check if service exists:")
      subprocess.run(["kubectl", "get", "svc", "-n", "dev"])
      print("\n2. Check if pods are running:")
      subprocess.run(["kubectl", "get", "pods", "-n", "dev"])

finally:
      print("\nStopping port-forward...")
      port_forward.terminate()
      port_forward.wait(timeout=5)

## Step 12: Deploy to Staging (Manual)

Repeat the process for staging environment.

In [ ]:
%%bash
# Create staging namespace
kubectl create namespace staging --dry-run=client -o yaml | kubectl apply -f -

# Apply manifests to staging
kubectl apply -f imperative/k8s/ -n staging

# Wait for deployment
kubectl rollout status deployment/imp-product-catalog -n staging --timeout=120s

In [ ]:
!kubectl get pods,svc,hpa -n staging

## Step 13: Deploy to Production (Manual)

Repeat the process for production environment.

In [ ]:
%%bash
# Create prod namespace
kubectl create namespace prod --dry-run=client -o yaml | kubectl apply -f -

# Apply manifests to prod
kubectl apply -f imperative/k8s/ -n prod

# Wait for deployment
kubectl rollout status deployment/imp-product-catalog -n prod --timeout=120s

In [ ]:
!kubectl get pods,svc,hpa -n prod

## Step 14: View Complete Deployment

Show resources across all environments.

In [ ]:
print("=== DEV ENVIRONMENT ===")
!kubectl get pods,hpa -n dev

print("\n=== STAGING ENVIRONMENT ===")
!kubectl get pods,hpa -n staging

print("\n=== PROD ENVIRONMENT ===")
!kubectl get pods,hpa -n prod

## Alternative: Use Dagger Pipeline

The included Dagger pipeline automates all the above steps.

In [ ]:
%%bash
# Run Dagger pipeline for dev environment
cd imperative
./deploy.sh dev v1.0.0-imperative

./deploy.sh staging v1.0.0-imperative

./deploy.sh prod v1.0.0-imperative


## Key Observations

### Traditional Approach Challenges:
- ❌ **746 lines** across 6 files
- ❌ **3 different tools**: Terraform, K8s, Dagger
- ❌ **Manual coordination** between tools
- ❌ **Context switching** between HCL, YAML, and Go
- ❌ **Terraform state** management complexity
- ❌ **Programming skills** required (Go for Dagger)
- ❌ **Manual multi-environment** deployments
- ❌ **No built-in approval gates** (need CI/CD)
- ❌ **Separate files** for HPA, Security, Resources

### What We Had to Do:
1. Write and maintain Terraform code (244 lines)
2. Write and maintain K8s manifests (196 lines)
3. Write and maintain Dagger pipeline (306 lines Go)
4. Manage Terraform state
5. Coordinate AWS credentials across tools
6. Manually deploy to each environment
7. Write custom testing logic in Go
8. No unified view of the application

### Compare to KubeVela:
- **258 lines** in 1 file vs 746 lines in 6 files
- **65% less code**
- **83% fewer files**
- **Single tool** vs 3 tools
- **Declarative YAML** vs imperative Go
- **Built-in workflow** with approval gates
- **Automated testing** in YAML
- **Policy-driven** environment config

## Cleanup

In [ ]:
%%bash
# Delete K8s resources
kubectl delete namespace dev staging prod

# Load AWS credentials
if [ -f "../../.env.aws" ]; then
    source ../../.env.aws
    export AWS_ACCESS_KEY_ID
    export AWS_SECRET_ACCESS_KEY
    export AWS_SESSION_TOKEN
    export AWS_DEFAULT_REGION
fi

# Empty S3 bucket (required before Terraform destroy)
set -a && source ../.env.aws && set +a && aws s3 rm s3://tenant-atlantis-product-images-imperative --recursive

# Destroy Terraform resources
cd imperative/terraform


if [ -f "../../../.env.aws" ]; then
    echo "Loading AWS credentials from .env.aws..."
    source ../../../.env.aws
    export AWS_ACCESS_KEY_ID
    export AWS_SECRET_ACCESS_KEY
    export AWS_SESSION_TOKEN
    export AWS_DEFAULT_REGION

    # Force Terraform to use environment variables
    unset AWS_PROFILE
    unset AWS_SDK_LOAD_CONFIG
    export AWS_CONFIG_FILE=/dev/null
    export AWS_SHARED_CREDENTIALS_FILE=/dev/null

    echo "✓ AWS credentials loaded"
    echo ""
else
    echo "Warning: .env.aws not found. AWS operations may fail."
    echo "Please create .env.aws in the kubecon-na-2025 directory"
    echo ""
fi

terraform destroy -auto-approve